In this example, we will analyze Wikipedia articles of different cities: Boston, Seattle, San Francisco, and more.

The below code snippet downloads the relevant data into files.

In [2]:
from pathlib import Path
import requests

wiki_titles = ["Toronto", "Seattle", "Chicago", "Boston", "Houston"]

for title in wiki_titles:
    response = requests.get(
        'https://en.wikipedia.org/w/api.php',
        params={
            'action': 'query',
            'format': 'json',
            'titles': title,
            'prop': 'extracts',
            # 'exintro': True,
            'explaintext': True,
        }
    ).json()
    page = next(iter(response['query']['pages'].values()))
    wiki_text = page['extract']

    data_path = Path('data')
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", 'w', encoding="utf-8") as fp:
        fp.write(wiki_text)

The next snippet loads all files into Document objects.

In [4]:
# Load all wiki documents
from llama_index import SimpleDirectoryReader
city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(input_files=[f"data/{wiki_title}.txt"]).load_data()

Defining the Set of Indexes

In [26]:
from llama_index import GPTVectorStoreIndex, ServiceContext, StorageContext, LLMPredictor
from langchain.llms.openai import OpenAIChat
import os

os.environ["OPENAI_API_KEY"] = 'Enter-API-KEY'
# set service context
llm_predictor_gpt4 = LLMPredictor(llm=OpenAIChat(temperature=0))#, model_name="gpt-4"
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor_gpt4, chunk_size_limit=1024
)

# Build city document index
vector_indices = {}
for wiki_title in wiki_titles:
    storage_context = StorageContext.from_defaults()
    # build vector index
    vector_indices[wiki_title] = GPTVectorStoreIndex.from_documents(
        city_docs[wiki_title], 
        service_context=service_context,
        storage_context=storage_context,
    )
    # set id for vector index
    vector_indices[wiki_title].index_struct.index_id = wiki_title
    # persist to disk
    storage_context.persist(persist_dir=f'./storage/{wiki_title}')

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 20744 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 16942 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 26082 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 18661 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_c

Querying a vector index lets us easily perform semantic search over a given city’s documents.

In [27]:
query_engine = vector_indices["Toronto"].as_query_engine()
response = query_engine.query("What are the sports teams in Toronto?")
print(str(response))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 8 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1835 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


Toronto is represented in five major league sports, with teams in the National Hockey League (NHL), Major League Baseball (MLB), National Basketball Association (NBA), Canadian Football League (CFL), and Major League Soccer (MLS). The specific teams are the Toronto Maple Leafs, Toronto Blue Jays, Toronto Raptors, Toronto Argonauts, and Toronto FC.


# # Defining a Graph for Compare/Contrast Queries

We will now define a composed graph in order to run compare/contrast queries. This graph contains a keyword table composed on top of existing vector indexes.

To do this, we first want to set the “summary text” for each vector index.

In [28]:
index_summaries = {}
for wiki_title in wiki_titles:
    # set summary text for city
    index_summaries[wiki_title] = (
        f"This content contains Wikipedia articles about {wiki_title}. "
        f"Use this index if you need to lookup specific facts about {wiki_title}.\n"
        "Do not use this index if you want to analyze multiple cities."
    )

Next, we compose a keyword table on top of these vector indexes, with these indexes and summaries, in order to build the graph.

In [32]:
from llama_index.indices.composability import ComposableGraph
from llama_index import GPTSimpleKeywordTableIndex
import nltk
nltk.download('stopwords')

graph = ComposableGraph.from_indices(
    GPTSimpleKeywordTableIndex,
    [index for _, index in vector_indices.items()], 
    [summary for _, summary in index_summaries.items()],
    max_keywords_per_chunk=50
)

# get root index
#root_index = graph.get_index( graph.index_struct.root_id,GPTSimpleKeywordTableIndex) #graph.index_struct.root_id,
# set id of root index
#root_index.set_index_id("compare_contrast")
root_summary = (
    "This index contains Wikipedia articles about multiple cities. "
    "Use this index if you want to compare multiple cities. "
)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\irfan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens


Querying this graph (with a query transform module), allows us to easily compare/contrast between different cities. An example is shown below.

In [48]:
# define decompose_transform
from llama_index.indices.query.query_transform.base import DecomposeQueryTransform
#from llama_index import llm_predictor_chatgpt
#from langchain import LLMPredictor
#,LLMPredictor,PromptHelper
decompose_transform = DecomposeQueryTransform(
    llm_predictor_gpt4, verbose=True
)

# define custom query engines
from llama_index.query_engine.transform_query_engine import TransformQueryEngine
custom_query_engines = {}
for index in vector_indices.values():
    query_engine = index.as_query_engine(service_context=service_context)
    query_engine = TransformQueryEngine(
        query_engine,
        query_transform=decompose_transform,
        transform_extra_info={'index_summary': index.index_struct.summary},
    )
    custom_query_engines[index.index_id] = query_engine
custom_query_engines[graph.root_id] = graph.root_index.as_query_engine(
    retriever_mode='simple',
    response_mode='tree_summarize',
    service_context=service_context,
)

# define query engine
graph_query_engine = graph.as_query_engine(custom_query_engines=custom_query_engines)

# query the graph
query_str = (
    "Compare and contrast the arts and culture of Houston and Boston. "
)
response_chatgpt = graph_query_engine.query(query_str)

INFO:llama_index.indices.keyword_table.retrievers:> Starting query: Compare and contrast the arts and culture of Houston and Boston. 
INFO:llama_index.indices.keyword_table.retrievers:query keywords: ['contrast', 'arts', 'culture', 'boston', 'houston', 'compare']
INFO:llama_index.indices.keyword_table.retrievers:> Extracted keywords: ['boston', 'houston']


> Current query: Compare and contrast the arts and culture of Houston and Boston. 
> New query: What are some notable cultural institutions or events in Boston?


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 11 tokens


> Current query: Compare and contrast the arts and culture of Houston and Boston. 
> New query: What are some notable cultural institutions or events in Boston?


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1894 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-XYjjwctakFxUFhYbmYWV6I9C on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.' error_param=None error_type=requests message='OpenAI API error received' stream_error=False
INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-XYjjwctakFxUFhYbmYWV6I9C on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. 

> Current query: Compare and contrast the arts and culture of Houston and Boston. 
> New query: What are some notable cultural institutions or events in Houston?


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 11 tokens
INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-XYjjwctakFxUFhYbmYWV6I9C on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.' error_param=None error_type=requests message='OpenAI API error received' stream_error=False
INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-XYjjwctakFxUFhYbmYWV6I9C on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add

> Current query: Compare and contrast the arts and culture of Houston and Boston. 
> New query: What are some notable cultural institutions or events in Houston?


INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-XYjjwctakFxUFhYbmYWV6I9C on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.' error_param=None error_type=requests message='OpenAI API error received' stream_error=False
INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-XYjjwctakFxUFhYbmYWV6I9C on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.' error_param=None error_type=requests message='Op

Defining the Unified Query Interface

In [49]:
from llama_index.tools.query_engine import QueryEngineTool

query_engine_tools = []

# add vector index tools
for wiki_title in wiki_titles:
    index = vector_indices[wiki_title]
    summary = index_summaries[wiki_title]
    
    query_engine = index.as_query_engine(service_context=service_context)
    vector_tool = QueryEngineTool.from_defaults(query_engine, description=summary)
    query_engine_tools.append(vector_tool)


# add graph tool
graph_description = (
    "This tool contains Wikipedia articles about multiple cities. "
    "Use this tool if you want to compare multiple cities. "
)
graph_tool = QueryEngineTool.from_defaults(graph_query_engine, description=graph_description)
query_engine_tools.append(graph_tool)

Now, we can define the routing logic and overall router query engine. Here, we use the LLMSingleSelector, which uses LLM to choose a underlying query engine to route the query to.

In [50]:
from llama_index.query_engine.router_query_engine import RouterQueryEngine
from llama_index.selectors.llm_selectors import LLMSingleSelector


router_query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(service_context=service_context),
    query_engine_tools=query_engine_tools
)

## Querying our Unified Interface

In [52]:
# ask a compare/contrast question 
response = router_query_engine.query(
    "Compare and contrast the arts and culture of Houston and Boston.",
)
print(str(response))

INFO:llama_index.query_engine.router_query_engine:Selecting query engine 5: This tool contains Wikipedia articles about multiple cities. Use this tool if you want to compare multiple cities..
INFO:llama_index.indices.keyword_table.retrievers:> Starting query: Compare and contrast the arts and culture of Houston and Boston.
INFO:llama_index.indices.keyword_table.retrievers:query keywords: ['contrast', 'arts', 'culture', 'boston', 'houston', 'compare']
INFO:llama_index.indices.keyword_table.retrievers:> Extracted keywords: ['boston', 'houston']


> Current query: Compare and contrast the arts and culture of Houston and Boston.
> New query: What are some notable cultural institutions or events in Boston?


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 11 tokens


> Current query: Compare and contrast the arts and culture of Houston and Boston.
> New query: What are some notable cultural institutions or events in Boston?


INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-XYjjwctakFxUFhYbmYWV6I9C on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.' error_param=None error_type=requests message='OpenAI API error received' stream_error=False
INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-XYjjwctakFxUFhYbmYWV6I9C on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.' error_param=None error_type=requests message='Op

> Current query: Compare and contrast the arts and culture of Houston and Boston.
> New query: What are some notable cultural institutions or events in Houston?


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 11 tokens
INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-XYjjwctakFxUFhYbmYWV6I9C on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.' error_param=None error_type=requests message='OpenAI API error received' stream_error=False
INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-XYjjwctakFxUFhYbmYWV6I9C on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add

> Current query: Compare and contrast the arts and culture of Houston and Boston.
> New query: What are some notable cultural institutions or events in Houston?


INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-XYjjwctakFxUFhYbmYWV6I9C on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.' error_param=None error_type=requests message='OpenAI API error received' stream_error=False
INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-XYjjwctakFxUFhYbmYWV6I9C on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.' error_param=None error_type=requests message='Op

Both Houston and Boston have a rich arts and culture scene with a variety of institutions and events. Boston is known for its symphony orchestra, ballet, opera company, and early music festival, while Houston is known for its rodeo, Greek festival, art car parade, and international festival. Both cities have museums of fine arts and natural science, as well as contemporary art museums. Boston has historic sites related to the American Revolution, while Houston has a theater district and a space center. Both cities also have pride parades and festivals. Overall, while there are some differences in the specific institutions and events, both Houston and Boston offer a diverse and vibrant arts and culture scene.


In [53]:
response = router_query_engine.query("What are the sports teams in Toronto?")
print(str(response))

INFO:llama_index.query_engine.router_query_engine:Selecting query engine 0: This content contains Wikipedia articles about Toronto. Use this index if you need to lookup specific facts about Toronto. Do not use this index if you want to analyze multiple cities..
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 8 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1835 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


Toronto is represented in five major league sports, with teams in the National Hockey League (NHL), Major League Baseball (MLB), National Basketball Association (NBA), Canadian Football League (CFL), and Major League Soccer (MLS). The specific teams are the Toronto Maple Leafs, Toronto Blue Jays, Toronto Raptors, Toronto Argonauts, and Toronto FC.
